In [184]:
# Import dependencies.
import sqlite3
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, text, inspect, func, MetaData, Table, Column, Integer, String, Float
from flask import Flask, render_template
import json

In [185]:
ss = pd.read_csv("spotify_songs.csv")

In [186]:
drop_ss = ss.dropna()

In [187]:
any_null_values = drop_ss.isnull().any()
#print(any_null_values)

In [188]:
#drop_ss.columns

In [189]:
rename_ss = drop_ss.rename(columns={'track_id': 'track_id', 
                           'track_name': 'track_name',
                           'track_artist': 'artist',
                           'track_popularity': 'popularity',
                           'track_album_id': 'album_id',
                           'track_album_name': 'album_name',
                           'track_album_release_date': 'album_release_date',
                           'playlist_name': 'playlist_name',
                           'playlist_id': 'playlist_id',
                           'playlist_genre': 'playlist_genre',
                           'playlist_subgenre': 'playlist_subgenre',
                           'danceability': 'danceability',
                           'energy': 'energy',
                           'key': 'key',
                           'loudness': 'loudness',
                           'mode': 'mode',
                           'speechiness': 'speechiness',
                           'acousticness': 'acousticness',
                           'instrumentalness': 'instrumentalness',
                           'liveness': 'liveness',
                           'valence': 'valence',
                           'tempo': 'tempo',
                           'duration_ms': 'duration_ms'})

In [190]:
#for index, row in rename_ss.iterrows():
    #track_artist = row['artist']
    #track_name = row['track_name']
    #album_name = row['album_name']
    #print(f'\nTrack Artist: {track_artist}\nTrack Name: {track_name}\nAlbum Name: {album_name}\n')

In [191]:
rename_ss['duration_seconds'] = rename_ss['duration_ms'] / 1000

In [192]:
rename_ss['minutes'], rename_ss['seconds'] = divmod(rename_ss['duration_seconds'], 60)

In [193]:
rename_ss['duration_minutes_seconds'] = rename_ss[['minutes', 'seconds']].apply(lambda x: '{:.0f}.{:02.0f}'.format(x[0], x[1]), axis=1)

In [194]:
rename_ss = rename_ss.drop(columns=["duration_seconds", "minutes", "seconds"])

In [195]:
rename_ss["duration_minutes_seconds"] = rename_ss["duration_minutes_seconds"].astype(float)

In [196]:
#rename_ss["duration_minutes_seconds"].dtype

In [197]:
column_types = rename_ss.dtypes
#print(column_types)

In [198]:
# creates a sqlite database and establishes a connection
conn = sqlite3.connect('spotify_songs.sqlite')

In [199]:
# reads the spotify_songs database (ss) into the sqlite database
rename_ss.to_sql("spotify_songs", conn, if_exists="replace")

32828

In [200]:
# commits changes to the sqlite database
conn.commit()

In [201]:
# closes the connection now that we are done with it
conn.close()

In [202]:
# creates a sqlite database engine using SQLAlchemy to allow SQLAlchemy to interact with the database 
engine = create_engine("sqlite:///spotify_songs.sqlite", echo=False)

In [205]:
tables = inspect(engine).get_table_names()
print(tables)

['spotify_songs']


In [206]:
columns = inspect(engine).get_columns('spotify_songs')
#for column in columns:
    #print(column['name'])

In [207]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [208]:
print(metadata.tables.keys())

dict_keys(['spotify_songs'])


In [209]:
# produces a Session class and creates an instance/object (session) to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

In [210]:
results = session.execute("SELECT * FROM spotify_songs LIMIT 10").fetchall()
#print(results)

In [211]:
Base = declarative_base()

In [212]:
class SpotifySongs(Base):
    __tablename__ = 'spotify_songs'
    track_id = Column(Integer, primary_key=True)
    track_name = Column(String(225))
    artist = Column(String(225))
    popularity = Column(Integer)
    album_id = Column(String(225))
    album_name = Column(String(225))
    album_release_date = Column(String(225))
    playlist_name = Column(String(225))
    playlist_id = Column(String(225))
    playlist_genre = Column(String(225))
    playlist_subgenre = Column(String(225))
    danceability = Column(Float)
    energy = Column(Float)
    key = Column(Integer)
    loudness = Column(Float)
    mode = Column(Integer)
    speechiness = Column(Float)
    acousticness = Column(Float)
    instrumentalness = Column(Float)
    liveness = Column(Float)
    valence = Column(Float)
    tempo = Column(Float)
    duration_ms = Column(Integer)
    duration_minutes_seconds = Column(Float)  

In [213]:
#Base.metadata.drop_all(engine)

In [214]:
results = session.query(SpotifySongs).all()

In [220]:
for row in results:
    if row.artist == "Black Sabbath":
        print(f'SONG: "{row.track_name}"\nALBUM: {row.album_name}\nLENGTH: {row.duration_minutes_seconds}\n')

SONG: "Changes"
ALBUM: The Ultimate Collection
LENGTH: 4.42

SONG: "Paranoid"
ALBUM: Greatest Hits
LENGTH: 2.48

SONG: "Iron Man - 2014 Remaster"
ALBUM: Paranoid (2014 Remaster)
LENGTH: 5.55

SONG: "Paranoid - 2016 Remaster"
ALBUM: Paranoid (2014 Remaster)
LENGTH: 2.48

SONG: "Iron Man"
ALBUM: Paranoid
LENGTH: 5.54

SONG: "Paranoid"
ALBUM: Paranoid (Remastered)
LENGTH: 2.48

SONG: "Paranoid"
ALBUM: Paranoid (2009 Remastered Version)
LENGTH: 2.48

SONG: "Paranoid"
ALBUM: Absolute Rock Anthems
LENGTH: 2.46

SONG: "Iron Man"
ALBUM: Lords Of Dogtown
LENGTH: 5.54

SONG: "Tomorrow's Dream"
ALBUM: Black Sabbath Vol. 4 (Remastered)
LENGTH: 3.09

SONG: "After Forever"
ALBUM: Master Of Reality (Remastered)
LENGTH: 5.26

SONG: "Looking for Today"
ALBUM: Sabbath Bloody Sabbath (Remastered)
LENGTH: 5.01

SONG: "It's Alright"
ALBUM: Technical Ecstasy (Remastered)
LENGTH: 3.59

SONG: "Heaven And Hell"
ALBUM: Heaven & Hell (Deluxe Edition)
LENGTH: 6.58

SONG: "Paranoid"
ALBUM: Paranoid
LENGTH: 2.48

S